In [33]:
import numpy as np
import sys
from tmp.Franke import SinglePathwayModel
from tmp.Franke import *
from tmp.Franke_feature import get_trace_features
import matplotlib.pyplot as plt
import os

In [2]:
dt = 2

In [3]:
pre_step, stim, response = get_data_pair_lchirp(1, 20000, DATATYPE=1, dt=dt)

In [4]:
true_feature = get_trace_features(response, dt=dt)

In [45]:
labels_params = [
            r'${sigmoid}_{HALF}$',
            r'${sigmoid}_{SLOPE}$',
            r'${sigmoid}_{MAX}$', r'$\tau_{TF}$',
            r'$g\_{INF}_k$', r'$g\_{INF}_k$',
            r'$g\_\tau_{HALF}$', r'$g\_\tau_{SLOPE}$',
            r'$g\_\tau_{MAX}$'
        ]

for i in range(3, 10):
    print("Processing {}".format(i))
    base_dir = './Result/exp_{}'.format(i)
    post_params_file = os.path.join(base_dir, 'posterior_samples.npy')
    
    params = np.load(post_params_file)

    fig = plt.figure(figsize=(20, 12))
    for i in range(params.shape[-1]):
        plt.subplot(3, 3, i + 1)
        plt.hist(params[:, i])
        plt.xlabel(labels_params[i])
    plt.savefig(os.path.join(base_dir, 'param_hist.png'), dpi=400)
    plt.clf()

    means_params = np.mean(params, axis=0)
    means_params

    model = SinglePathwayModel(means_params, dt=dt, single=True)
    res = model.run(stim)[0][pre_step:]
    feature = get_trace_features(res, dt=dt)

    plt.plot(response, c='b', label='True')
    plt.plot(res, c='r', label='Prediction')
    plt.legend()
    plt.savefig(os.path.join(base_dir, 'response_contrast.png'), dpi=400)
    plt.clf()

    plt.plot(true_feature, c='b', label='True')
    plt.plot(feature, c='r', label='Prediction')
    plt.legend()
    plt.savefig(os.path.join(base_dir, 'feature_contrast.png'), dpi=400)
    plt.clf()


Processing 3
Processing 4
Processing 5
Processing 6
Processing 7
Processing 8
Processing 9


<Figure size 1440x864 with 0 Axes>

<Figure size 1440x864 with 0 Axes>

<Figure size 1440x864 with 0 Axes>

<Figure size 1440x864 with 0 Axes>

<Figure size 1440x864 with 0 Axes>

<Figure size 1440x864 with 0 Axes>

<Figure size 1440x864 with 0 Axes>

In [27]:
x = [np.array([[0.5, 4]]),  # half of sigmoid
     np.array([[0.1, 3]]), # slope of sigmoid
     np.array([[0.1, 5]]), # maximum of sigmoid
     np.array([[0.1, 50]]), # tau fo temporal filter
     np.array([[0.01, 2]]), # k of gain infi
     np.array([[0.2, 3]]), # m of gain infi
     np.array([[-2, +2]]), # half of gain tau
     np.array([[-1, +1]]), # slope of gain tau
     np.array([[100, 1500]])] # max of gain tau

In [31]:
_iter = int(1000)
for _ in range(_iter):
    param_list = list()
    for param_range in x:
        theta = np.random.uniform(*param_range[0])
        param_list.append(theta)
    print("Eproch {} param {}".format(_, param_list))
    model = SinglePathwayModel(param_list, dt=dt, single=True)
    res = model.run(stim)[0][pre_step:]
    feature = get_trace_features(res, dt=dt)
    assert feature.shape[0] == 218

Eproch 0 param [0.5071047830111993, 0.10366093248526272, 0.1021618931297103, 0.11463167229906629, 0.01946876816122154, 0.22061991623461116, -1.4328061271244685, -0.8000586087539993, 122.65501890836353]
Eproch 1 param [0.5036222660624434, 0.1096128528766442, 0.10246075015009916, 0.1010230946496341, 0.018217060998755225, 0.2551149046545783, -1.837457614530423, -0.494686090461131, 122.0348595798901]
Eproch 2 param [0.5010517518871126, 0.10852289874429272, 0.10871555909860214, 0.10765511765482785, 0.01036778927944026, 0.20796140895844395, -1.3805518620520227, -0.5901765246185442, 143.30249120502458]
Eproch 3 param [0.500250509926359, 0.10962300170942862, 0.11040566306665713, 0.11671175432390293, 0.016258228261279414, 0.2645566072404549, -1.3774642111571826, -0.9849377552708735, 107.69686035674529]
Eproch 4 param [0.5028469998963155, 0.10200784630884287, 0.11785164741433989, 0.1074524150841191, 0.01820822496310849, 0.22586667210267594, -1.67225304441325, -0.7211547264081017, 109.73137382892

Eproch 41 param [0.5003828432121102, 0.11099202186169319, 0.10464199533699721, 0.10721346667882757, 0.011249013920965195, 0.2645841310806254, -1.5873192151427582, -0.4551319205983324, 148.47299948523153]
Eproch 42 param [0.5020581084983868, 0.10417476260717042, 0.11374871143201905, 0.10875714552313774, 0.01545530791155417, 0.2273184494831185, -1.530557756062946, -0.6010436755952693, 125.59176033877338]
Eproch 43 param [0.5043352851922223, 0.11223071055038525, 0.11905093623847543, 0.11201863340817453, 0.01659159668942535, 0.23686391604778756, -1.4689873015163672, -0.5603349057755009, 138.04274517862032]
Eproch 44 param [0.5038547815235702, 0.10665469229961351, 0.1101279180855765, 0.11135224149971439, 0.012530420559313055, 0.2648104805759637, -1.276271475920879, -0.535927909931116, 145.266195652386]
Eproch 45 param [0.5062036045071531, 0.10335945764916443, 0.10051554231096671, 0.10740707756013616, 0.017661584572034594, 0.2397190835066813, -1.2806916012637775, -0.3549600468755624, 116.646

KeyboardInterrupt: 

In [5]:
x_list = list()
for i in range(int(1e4)):
    x = np.random.random(response.shape)
    x_list.append(x)
    get_trace_features(x, dt=dt).shape

/home/skw/anaconda3/envs/py3.6/lib/python3.6/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [6]:
x = np.concatenate(x_list)

In [ ]:
x_list[12]

In [ ]:
feature_x = get_trace_features(x_list[12], dt=dt)
feature_x.shape

In [ ]:
feature_x

In [ ]:
plt.plot(x * 100)

In [ ]:
np.save('wrong_trace.npy', x)

In [ ]:
response.shape

In [ ]:
pre_step

In [ ]:
plt.plot(stim, c='b')

In [ ]:
plt.plot(response, c='r')

In [ ]:
model = SinglePathwayModel(np.random.random((9)), dt=dt, single=True)

In [ ]:
random_response = model.run(stim)

In [ ]:
from delfi.simulator.BaseSimulator import BaseSimulator

class Ribon(BaseSimulator):
    def __init__(self, I, dt, dim_param, pre_step, Model, seed=None):
        """Ribon simulator

        Parameters
        ----------
        I : array
            Numpy array with the input current
        dt : float
            Timestep
        dim_param : int
            Number of parameter
        pre_step : int
            Number of time points before the stimuli
        seed : int or None
            If set, randomness across runs is disabled
        """

        super().__init__(dim_param=dim_param, seed=seed)
        self.I = I
        self.dt = dt
        self.t = np.arange(0, len(self.I), 1)*self.dt
        self.RibonModel = Model
        self.pre_step = pre_step

    def gen_single(self, params):
        """Forward model for simulator for single parameter set

        Parameters
        ----------
        params : list or np.array, 1d of length dim_param
            Parameter vector

        Returns
        -------
        dict : dictionary with data
            The dictionary must contain a key data that contains the results of
            the forward run. Additional entries can be present.
        """
        params = np.asarray(params)

        assert params.ndim == 1, 'params.ndim must be 1'

        hh_seed = self.gen_newseed()

        model = self.RibonModel(params, dt=self.dt, single=True)
        states = model.run(self.I)[0][pre_step:]

        return {'data': states.reshape(-1),
                'time': self.t,
                'dt': self.dt,
                'I': self.I}

In [ ]:
param_dim = 9
RibonSimulator = Ribon(stim, dt, param_dim, pre_step, SinglePathwayModel)

In [ ]:
data = RibonSimulator.gen_single(np.random.random((9)))

In [ ]:
plt.plot(data['data'])

In [ ]:
from delfi.summarystats.BaseSummaryStats import BaseSummaryStats
from scipy import stats as spstats

class RibonStats(BaseSummaryStats):
    """Moment based SummaryStats class for the Ribon model

    Calculates summary statistics
    """
    def __init__(self, t_on, t_off, dt, n_summary, seed=None):
        """See SummaryStats.py for docstring"""
        super(RibonStats, self).__init__(seed=seed)
        self.t_on = t_on
        self.t_off = t_off
        self.dt = dt
        self.n_summary = n_summary

    def calc(self, repetition_list):
        """Calculate summary statistics

        Parameters
        ----------
        repetition_list : list of dictionaries, one per repetition
            data list, returned by `gen` method of Simulator instance

        Returns
        -------
        np.array, 2d with n_reps x n_summary
        """
        stats = []
        for r in range(len(repetition_list)):
            x = repetition_list[r]
            # sum_stats_vec = get_trace_features(x['data'], self.dt)
            sum_stats_vec = x['data']
            stats.append(sum_stats_vec)

        return np.asarray(stats)

In [46]:
x = [np.array([[0.5, 4]]),  # half of sigmoid
     np.array([[0.1, 3]]), # slope of sigmoid
     np.array([[0.1, 5]]), # maximum of sigmoid
     np.array([[0.1, 50]]), # tau fo temporal filter
     np.array([[0.01, 2]]), # k of gain infi
     np.array([[0.2, 3]]), # m of gain infi
     np.array([[-2, +2]]), # half of gain tau
     np.array([[-1, +1]]), # slope of gain tau
     np.array([[100, 1500]])] # max of gain tau

In [47]:
import delfi.distribution as dd

seed_p = 2
prior_min = np.concatenate(x, axis=0)[:, 0]
prior_max = np.concatenate(x, axis=0)[:, 1]
prior = dd.Uniform(lower=prior_min, upper=prior_max,seed=seed_p)

In [ ]:
import delfi.generator as dg

# input current, time step
t_on = 0
t_off = len(stim)

# seeds
seed_m = 1

# summary statistics hyperparameters
n_summary = 218

# define model, prior, summary statistics and generator classes
s = RibonStats(t_on=t_on, t_off=t_off, dt=dt, n_summary=n_summary)

n_processes = 5

seeds_m = np.arange(1, n_processes+1, 1)
m = []
for i in range(n_processes):
    m.append(Ribon(stim, dt, param_dim, pre_step, SinglePathwayModel, seed=seeds_m[i]))
g = dg.MPGenerator(models=m, prior=prior, summary=s)

In [ ]:
seed_inf = 1

pilot_samples = 10

# training schedule
n_train = 10
n_rounds = 1

# fitting setup
minibatch = 10
epochs = 10
val_frac = 0.05

# network setup
n_hiddens = [50,50]

# convenience
prior_norm = True

# MAF parameters
density = 'maf'
n_mades = 5         # number of MADES

In [ ]:
import delfi.inference as infer

# inference object
res = infer.SNPEC(g,
                obs=response,
                n_hiddens=n_hiddens,
                seed=seed_inf,
                pilot_samples=pilot_samples,
                n_mades=n_mades,
                prior_norm=prior_norm,
                density=density)

In [ ]:
# train
log, _, posterior = res.run(
                    n_train=n_train,
                    n_rounds=n_rounds,
                    minibatch=minibatch,
                    epochs=epochs,
                    silent_fail=False,
                    proposal='prior',
                    val_frac=val_frac,
                    verbose=True,)

In [ ]:
import delfi

In [ ]:
dir(delfi.)

In [ ]:
log

In [ ]:
fig = plt.figure(figsize=(15,5))

plt.plot(log[0]['loss'],lw=2)
plt.xlabel('iteration')
plt.ylabel('loss');

In [ ]:
from delfi.utils.viz import samples_nd

prior_min = g.prior.lower
prior_max = g.prior.upper
prior_lims = np.concatenate((prior_min.reshape(-1,1),prior_max.reshape(-1,1)),axis=1)

posterior_samples = posterior[0].gen(10000)

###################
# colors
hex2rgb = lambda h: tuple(int(h[i:i+2], 16) for i in (0, 2, 4))

# RGB colors in [0, 255]
col = {}
col['GT']      = hex2rgb('30C05D')
col['SNPE']    = hex2rgb('2E7FE8')
col['SAMPLE1'] = hex2rgb('8D62BC')
col['SAMPLE2'] = hex2rgb('AF99EF')

# convert to RGB colors in [0, 1]
for k, v in col.items():
    col[k] = tuple([i/255 for i in v])

labels_params = [ str(i) for i in range(9)]
###################
# posterior
fig, axes = samples_nd(posterior_samples,
                       limits=prior_lims,
                       ticks=prior_lims,
                       labels=labels_params,
                       fig_size=(5,5),
                       diag='kde',
                       upper='kde',
                       hist_diag={'bins': 50},
                       hist_offdiag={'bins': 50},
                       kde_diag={'bins': 50, 'color': col['SNPE']},
                       kde_offdiag={'bins': 50},
                       # points=[true_params],
                       points_offdiag={'markersize': 5},
                       points_colors=[col['GT']],
                       title='');


